# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [2]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\L

## Load the dataset

In [3]:
dataset = 'cifar10' #'cifar10' #'imagenet'
splitsize = 5 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset= dataset, dataset_split=splitsize, root='../data')

Files already downloaded and verified


## Load and Prepare Adversarial Model

In [4]:
model = 'standard' # CIFAR10: (MainiAVG, MainiMSD, CroceL1, corruption_robust, standard, Ding2020MMA, Wang2023Better_WRN-28-10), IN: (standard_resnet50, Salman2020Do_R50)
net, art_net, fb_net, alias = utils.get_model(dataset= dataset, modelname=model, norm='Linf') # specify the norm in case of robustbench models

c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\utils.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'./models/pretrained_models/{mo

## Test Accuracy

In [5]:
utils.test_accuracy(net, xtest, ytest)
#print(ytest)


Accuracy of the test set is: 100.000%



## Parameters 

In [6]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12 if dataset == "cifar10" else 75, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations = 300, 
                        save_images = 1,
                        verbose=True)

# Hyperparameter Sweep Experiment

In [7]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[1.0]
hyperparameter = 'learning_rate' #'beta'

#results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

In [8]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                'brendel_bethge',
                #'pointwise_blackbox',
                #'geoda_blackbox',
                #'sparse_rs_blackbox',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'exp_attack_l1',
                'custom_apgd'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)

		-------------------------- Processing Attack: brendel_bethge --------------------------

Image 0		Adversarial_distance (L1 / L2): 0.9184 / 0.38220		Runtime: 92.749482 seconds


c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\eagerpy\tensor\pytorch.py:263: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  return type(self)(torch.as_tensor(a, device=self.raw.device))


Image 1		Adversarial_distance (L1 / L2): 2.8349 / 0.77659		Runtime: 24.774701 seconds
Image 2		Adversarial_distance (L1 / L2): 1.5573 / 0.39268		Runtime: 24.520703 seconds
Image 3		Adversarial_distance (L1 / L2): 0.4448 / 0.21534		Runtime: 29.912039 seconds
Image 4		Adversarial_distance (L1 / L2): 2.4337 / 0.78943		Runtime: 23.056810 seconds

Adversarial accuracy (L1 / L2/ EN): 0.00 / 40.00/ 0.00%


average sparsity: 99.06%


Total runtime:  195.0137 seconds

attack success rate in epsilon (L1 / L2):  100.0  /  60.0
mean adv. distance (L1 / L2):  1.6378  /  0.5112
		-------------------------- Processing Attack: exp_attack_l1 --------------------------



ExpAttackL1: 100%|██████████| 1/1 [00:24<00:00, 25.00s/it]


Image 0		Adversarial_distance (L1 / L2): 7.7009 / 1.19904		Runtime: 25.043595 seconds


...

Image 2		Adversarial_distance (L1 / L2): 11.3303 / 1.21442		Runtime: 27.833011 seconds


ExpAttackL1: 100%|██████████| 1/1 [00:27<00:00, 27.31s/it]


Image 3		Adversarial_distance (L1 / L2): 5.1302 / 1.13916		Runtime: 27.356316 seconds


ExpAttackL1: 100%|██████████| 1/1 [00:29<00:00, 29.16s/it]


Image 4		Adversarial_distance (L1 / L2): 11.1600 / 1.26811		Runtime: 29.243632 seconds

Adversarial accuracy (L1 / L2/ EN): 0.00 / 100.00/ 0.00%


average sparsity: 96.02%


Total runtime:  136.3926 seconds

attack success rate in epsilon (L1 / L2):  100.0  /  0.0
mean adv. distance (L1 / L2):  9.2993  /  1.2026
		-------------------------- Processing Attack: custom_apgd --------------------------

using custom version including apgd-ce.
initial accuracy: 100.00%
apgd-ce - 1/1 - 1 out of 1 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 20.4 s)
max L1 perturbation: 11.99999, nan in tensor: 0, max: 1.00000, min: 0.06098
robust accuracy: 0.00%
Image 0		Adversarial_distance (L1 / L2): 12.0000 / 0.97221		Runtime: 20.841712 seconds
using custom version including apgd-ce.
initial accuracy: 100.00%
apgd-ce - 1/1 - 1 out of 1 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 15.1 s)
max L1 perturbation: 12.00000, nan in tensor: 0, max: 0.98824, min